In [28]:
import pandas as pd
import math
import scipy.stats as st
from sklearn.preprocessing import MinMaxScaler

In [3]:
# Veriyi okuduk.
df = pd.read_csv("amazon_review.csv")

In [7]:
# Adım 1: Ürünün ortalama puanını hesaplayınız.
df["overall"].mean()

4.587589013224822

In [ ]:
# Adım 2: Tarihe göre ağırlıklı puan ortalamasını hesaplayınız

# reviewTime değişkenini tarih değişkeni olarak tanıt
df["reviewTime"] = pd.to_datetime(df["reviewTime"])

In [29]:
# reviewTime'ın max değerini current_date olarak kabul etmek.
current_date = df["reviewTime"].max()

In [ ]:
# her bir puan-yorum tarihi ile current_date'in farkını gün cinsinden ifade ederek yeni değişken oluşturmanız ve gün cinsinden ifade edilen 
# değişkeni quantile fonksiyonu ile 4'e bölüp (3 çeyrek verilirse 4 parça çıkar) çeyrekliklerden gelen değerlere göre ağırlıklandırma yapmanız 
# gerekir. Örneğin q1 = 12 ise ağırlıklandırırken 12 günden az süre önce yapılan yorumların ortalamasını alıp bunlara yüksek ağırlık vermek gibi

In [30]:
df["day_diff"] = (current_date - df["reviewTime"]).dt.days

In [31]:
df["day_diff"].quantile([.25, .5, .75]) 

0.25    280.0
0.50    430.0
0.75    600.0
Name: day_diff, dtype: float64

In [32]:
# Adım 3: Ağırlıklandırılmış puanlamada her bir zaman diliminin ortalamasını karşılaştırıp yorumlayınız

print(df.loc[(df["day_diff"] <= 280), "overall"].mean())
print(df.loc[(df["day_diff"] > 280) & (df["day_diff"] <= 430), "overall"].mean())
print(df.loc[(df["day_diff"] > 430) & (df["day_diff"] <= 600), "overall"].mean()) 
print(df.loc[(df["day_diff"] > 600), "overall"].mean())

4.6957928802588995
4.636140637775961
4.571661237785016
4.4462540716612375


In [ ]:
# Görev 2: Ürün için ürün detay sayfasında görüntülenecek 20 review’i belirleyiniz

In [35]:
# Adım 1: helpful_no değişkenini üretiniz

# total_vote bir yoruma verilen toplam up-down sayısıdır.
# up, helpful demektir.
# Veri setinde helpful_no değişkeni yoktur, var olan değişkenler üzerinden üretilmesi gerekmektedir.
# Toplam oy sayısından (total_vote) yararlı oy sayısı (helpful_yes) çıkarılarak yararlı bulunmayan oy sayılarını (helpful_no) bulunuz.

df["helpful_no"] = df["total_vote"] - df["helpful_yes"]

In [ ]:
# Adım 2: score_pos_neg_diff, score_average_rating ve wilson_lower_bound skorlarını hesaplayıp veriye ekleyiniz

# score_pos_neg_diff, score_average_rating ve wilson_lower_bound skorlarını hesaplayabilmek için score_pos_neg_diff, 
# score_average_rating ve wilson_lower_bound fonksiyonlarını tanımlayınız.

# score_pos_neg_diff'a göre skorlar oluşturunuz. Ardından; df içerisinde score_pos_neg_diff ismiyle kaydediniz.
# score_average_rating'a göre skorlar oluşturunuz. Ardından; df içerisinde score_average_rating ismiyle kaydediniz.
# wilson_lower_bound'a göre skorlar oluşturunuz. Ardından; df içerisinde wilson_lower_bound ismiyle kaydediniz

In [36]:
# score_pos_nef_diff
def score_up_down_diff(up, down):
    return up - down
df["score_pos_neg_diff"] = score_up_down_diff(df["helpful_yes"], df["helpful_no"])

In [37]:
# score_average_rating
def score_average_rating(up, down):
    if up + down == 0:
        return 0
    return up / (up + down)

In [38]:
# wilson_lower_bound
def wilson_lower_bound(up, down, confidence=0.95):
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)
df["wilson_lower_bound"] = df.apply(lambda x: wilson_lower_bound(x["helpful_yes"], x["helpful_no"]), axis=1)

In [39]:
# Adım 3: 20 Yorumu belirleyiniz ve sonuçları Yorumlayınız
# wilson_lower_bound'a göre ilk 20 yorumu belirleyip sıralayanız

df.sort_values("wilson_lower_bound", ascending=False).head(20)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,days_since_review,weighted_review,helpful_no,score_pos_neg_diff,wilson_lower_bound
2031,A12B7ZMXFI6IXY,B007WTAJTO,"Hyoun Kim ""Faluzure""","[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.0,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1367366400,2013-01-05,701,1952,2020,701,-5.0,68,1884,0.957544
3449,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.0,Top of the class among all (budget-priced) mic...,1348617600,2012-09-26,802,1428,1505,802,-5.0,77,1351,0.936519
4212,AVBMZZAFEKO58,B007WTAJTO,SkincareCEO,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.0,1 Star reviews - Micro SDXC card unmounts itse...,1375660800,2013-05-08,578,1568,1694,578,1.0,126,1442,0.912139
317,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.0,"Warning, read this!",1346544000,2012-02-09,1032,422,495,1032,-1.0,73,349,0.818577
4672,A2DKQQIZ793AV5,B007WTAJTO,Twister,"[45, 49]",Sandisk announcement of the first 128GB micro ...,5.0,Super high capacity!!! Excellent price (on Am...,1394150400,2014-07-03,157,45,49,157,-5.0,4,41,0.808109
1835,A1J6VSUM80UAF8,B007WTAJTO,goconfigure,"[60, 68]",Bought from BestBuy online the day it was anno...,5.0,I own it,1393545600,2014-02-28,282,60,68,282,0.0,8,52,0.784651
3981,A1K91XXQ6ZEBQR,B007WTAJTO,"R. Sutton, Jr. ""RWSynergy""","[112, 139]",The last few days I have been diligently shopp...,5.0,"Resolving confusion between ""Mobile Ultra"" and...",1350864000,2012-10-22,776,112,139,776,-5.0,27,85,0.732136
3807,AFGRMORWY2QNX,B007WTAJTO,R. Heisler,"[22, 25]",I bought this card to replace a lost 16 gig in...,3.0,"Good buy for the money but wait, I had an issue!",1361923200,2013-02-27,648,22,25,648,-3.0,3,19,0.700442
4306,AOHXKM5URSKAB,B007WTAJTO,Stellar Eller,"[51, 65]","While I got this card as a ""deal of the day"" o...",5.0,Awesome Card!,1339200000,2012-09-06,822,51,65,822,-5.0,14,37,0.670334
4596,A1WTQUOQ4WG9AI,B007WTAJTO,"Tom Henriksen ""Doggy Diner""","[82, 109]",Hi:I ordered two card and they arrived the nex...,1.0,Designed incompatibility/Don't support SanDisk,1348272000,2012-09-22,806,82,109,806,-1.0,27,55,0.663595
